# Introduction to Gemini

The Gemini API can be used to generate many different outputs such as text, images, video and audio. In this tutorial, we'll go through text generation and check some metadata and tokens

## Text generation

In [7]:
from google import genai

# The client gets the API key from the environment variable `GEMINI_API_KEY`.
client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Generate some funny jokes about data engineering. Give 5 points",
)
print(response.text)

Here are 5 funny jokes about data engineering:

1.  **A data engineer, a data scientist, and a business analyst walk into a bar.** The data engineer immediately asks, "Is the tap data clean? And how often do you refresh the ice bucket?"
2.  **What's a data engineer's favorite bedtime story?** The one about the pipeline that ran successfully on the first try, with no schema changes, and then worked perfectly in production for a year.
3.  **A business stakeholder asks a data engineer for "real-time insights."** The data engineer smiles and says, "Great! How real-time are we talking? Is a 24-hour latency considered 'real-enough' time for you?"
4.  **Why did the data engineer get kicked out of the library?** Because they kept trying to enforce a strict schema on the Dewey Decimal System.
5.  **My therapist told me to identify my triggers.** I just showed them my Airflow dashboard.


In [58]:
response.__dict__.keys()

dict_keys(['sdk_http_response', 'candidates', 'create_time', 'model_version', 'prompt_feedback', 'response_id', 'usage_metadata', 'automatic_function_calling_history', 'parsed'])

### Analyze tokens

Tokens are the basic units of text for LLMs used to process and generate language. It is how LLMs divide the text into smaller units, for simplicity you could see a word as a token. Tokens are also what you pay for when you use the APIs

The free tier in gemini API allows for (Gemini 2.5 flash)

- Requests per minute (RPM): 10
- Tokens per minute (TPM): 250 000
- Requests per day (RDP): 250

as of 2025-11-11

[Check documentation here for latest limits](https://ai.google.dev/gemini-api/docs/rate-limits)

There is a possiblity to upgrade to higher tiers, which allows for more generous rate limits, but comes with higher costs. 

In [29]:
metadata = response.usage_metadata
metadata

GenerateContentResponseUsageMetadata(
  candidates_token_count=262,
  prompt_token_count=13,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=13
    ),
  ],
  thoughts_token_count=972,
  total_token_count=1247
)

In [30]:
from pydantic import BaseModel

# GenerateContentResponseUsageMetadata is a pydantic model, which means we can
# use dot operator to get different attributes
isinstance(metadata, BaseModel)

True

In [31]:
print("Output tokens - number of tokens in models response")
print(f"{metadata.candidates_token_count = }")

Output tokens - number of tokens in models response
metadata.candidates_token_count = 262


In [32]:
print("Tokens in user input")
print(f"{metadata.prompt_token_count = }")

Tokens in user input
metadata.prompt_token_count = 13


In [ ]:
# a lot of tokens used here
print("Tokens used for internal thinking")
print(f"{metadata.thoughts_token_count = }")

Tokens used for internal thinking
metadata.thoughts_token_count = 972


In [ ]:
print("Total tokens used - this is the billing number")
print(f"{metadata.total_token_count = }")

Tokens used for internal thinking
metadata.total_token_count = 1247


### Thinking 

- we can see that the thinking process takes a lot of tokens and also takes long time, so if you want answers quicker but chooses to have less thinking you can set the `thinking budget`


In [44]:
from google.genai import types

# The client gets the API key from the environment variable `GEMINI_API_KEY`.
client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Generate some funny jokes about data engineering. Give 5 points",
    config=types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(thinking_budget=0)
    ),
)
print(response.text)

Here are 5 funny jokes about data engineering:

1. **Why did the data engineer break up with their partner?** Because they kept bringing up old, unstructured data from their past, and the engineer just couldn't process it anymore.

2. **What's a data engineer's favorite type of music?** Anything with a good *flow*... especially if it's orchestrated.

3. **My doctor told me I needed to reduce my stress, so I became a data engineer.** Now, instead of worrying about my own problems, I worry about data pipelines failing at 3 AM. Much better!

4. **A data engineer walks into a bar.** The bartender asks, "What can I get for you?" The engineer replies, "Just give me all your raw ingredients, and I'll figure out how to make a beer that actually works tomorrow morning... maybe."

5. **You know you're a data engineer when...** you see a beautifully organized spreadsheet and your first thought isn't "Wow," but "I bet I could automate that into a robust, scalable ELT process."


In [61]:
print(repr(response.usage_metadata))

print("Ah much cheaper, but is the result as good as the thinking?")

GenerateContentResponseUsageMetadata(
  candidates_token_count=233,
  prompt_token_count=13,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=13
    ),
  ],
  total_token_count=246
)
Ah much cheaper, but is the result as good as the thinking?


## System instructions

In [71]:
system_instruction = """You are a joking robot called Ro Båt, which 
        will always answer with a programming joke.
        """

prompt = "What is the weather today?"

response = client.models.generate_content(
    model="gemini-2.5-flash",
    config=types.GenerateContentConfig(
        system_instruction=system_instruction,
    ),
    contents=prompt,
)

response.text

"ERROR 404: Weather data not found! It seems my forecast module is encountering a few bugs and keeps returning a 'Segmentation Fault: Core Dump' every time I try to debug the precipitation levels. Might need a patch!"

In [74]:
response.usage_metadata

GenerateContentResponseUsageMetadata(
  candidates_token_count=47,
  prompt_token_count=32,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=32
    ),
  ],
  thoughts_token_count=688,
  total_token_count=767
)

In [81]:
print(f"{len(system_instruction.split()) = }")
print(f"{len(prompt.split()) = }")
print("plus some formatting overhead")

print(f"prompt token count {response.usage_metadata.prompt_token_count}")

len(system_instruction.split()) = 16
len(prompt.split()) = 5
plus some formatting overhead
prompt token count 32


### Temperature

In [91]:
story_prompt = "write a 2 sentence story about a gray rabbit"

boring_story = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=story_prompt,
    config=types.GenerateContentConfig(temperature=0.0),
)

print(boring_story.text)

A small gray rabbit, its fur the color of twilight, twitched its nose, sampling the scent of fresh clover. With a flick of its ears, it hopped deeper into the meadow, a tiny shadow against the rising sun.


In [ ]:

# you can see that the outputs are similar to the first when temperature is 0
boring_story = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=story_prompt,
    config=types.GenerateContentConfig(temperature=0.0),
)

print(boring_story.text)

A small gray rabbit, its fur the color of a stormy sky, twitched its nose, sniffing the damp morning air for tender shoots. With a sudden flick of its ears, it vanished into the dense undergrowth, a silent gray blur against the green.


In [93]:

creative_story = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=story_prompt,
    config=types.GenerateContentConfig(temperature=2.0),
)

print(creative_story.text)

Dusty, a small gray rabbit, sat absolutely still in the tall grass, his twitching nose the only sign of life. A loud snap of a twig nearby sent him bolting like a silver blur straight into the safety of his burrow.


In [ ]:
# very different story
creative_story = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=story_prompt,
    config=types.GenerateContentConfig(temperature=2.0),
)

print(creative_story.text)

A small, gray rabbit cautiously emerged from its burrow, twitching its nose at the fresh morning dew. It hopped quietly across the lawn, ever watchful for hawks, until it found a patch of juicy dandelions to nibble.


## Multimodal inputs

In [ ]:
client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents={
        "parts": [
            {"text": "Tell me about this dude"},
            {
                "inline_data": {
                    "mime_type": "image/png",
                    "data": open("assets/kokchun.png", "rb").read(),
                }
            },
        ]
    },
)
print(response.text)

Based on the image, here's a description of this "dude":

**Appearance:**
*   He is a young adult male, likely in his 20s or early 30s, depicted in a cartoon style.
*   He has short, dark, slightly spiky hair.
*   He wears rectangular, black-rimmed glasses, which often suggest a studious or intellectual personality.
*   His skin tone is light to medium.
*   He has a neutral to slightly serious or contemplative expression on his face, with a straight mouth and slightly furrowed brows, suggesting focus or deep thought.

**Clothing:**
*   He is wearing a raglan-style shirt (also known as a baseball tee) with black sleeves and an off-white/tan body.
*   The most defining feature is his shirt graphic, which displays a humorous statistical pun:
    *   Above a classic bell curve (representing a normal distribution), it reads "NORMAL DISTRIBUTION."
    *   Below, above a ghost-shaped curve (which also somewhat resembles a bell curve), it reads "PARANORMAL DISTRIBUTION."

**Activity and Settin